In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
from talib import RSI, ATR, BBANDS

from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *


C:\Users\ASUS\AppData\Local\Temp\ipykernel_6164\258418730.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
test = DataLoader(ticker='NVDA', start='2022-01-01', end='2024-12-31', freq='1d', test_size=0.5)
test.run()

df = test.data['test_data']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [15]:
class MomentumReversal(Strategy):
    rsi_period = 3
    ma_period = 10
    atr_multiplier = 1.5

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)
        self.ma = self.I(SMA, self.data.Close, self.ma_period)
        self.atr = self.I(ATR, self.data.High, self.data.Low, self.data.Close, timeperiod=14)

    def next(self):
        if self.rsi[-1] < 30 and self.data.Close[-1] < self.ma[-1]:
            self.buy(sl=self.data.Close[-1] - self.atr_multiplier * self.atr[-1], tag='hello')
        elif self.rsi[-1] > 60:
            self.position.close()

class SidewaysSwing(Strategy):
    bb_period = 20
    atr_multiplier = 1.5

    def init(self):
        self.upper, self.middle, self.lower = self.I(lambda x: BBANDS(x, timeperiod=self.bb_period)[:3], self.data.Close)

    def next(self):
        if self.data.Close[-1] <= self.lower[-1]:
            self.buy(sl=self.lower[-1] - self.atr_multiplier * self.data.ATR[-1])
        elif self.data.Close[-1] >= self.upper[-1]:
            self.position.close()

class MSFT102(Strategy):

    def init(self):
        pass

    def buyLimitPartialTP(self, price: float, tp_price: float, tp_partial_price: float, size: float, tp_partial_size_perce: float):
        tp_partial_size = size*tp_partial_size_perce
        tp_size         = size-tp_partial_size

        self.buy(limit=price,  size=tp_size, tag='Hello')
        self.buy(limit=price, size=tp_partial_size, tag='Hello2')

    def next(self):
        self.buyLimitPartialTP(price=100, tp_price=105, tp_partial_price=102.5, size=20, tp_partial_size_perce=0.5)

# Load data (Using GOOG sample data from backtesting.py)

# Run backtest for Momentum Reversal
bt1 = Backtest(df, MSFT102, cash=10_000, commission=0.001)
stats1 = bt1.run()
bt1.plot()

# Run backtest for Sideways Swing
# bt2 = Backtest(df, SidewaysSwing, cash=10_000, commission=0.001)
# stats2 = bt2.run()
# bt2.plot()

print("Momentum Reversal Strategy:\n", stats1)
# print("Sideways Swing Strategy:\n", stats2)


Momentum Reversal Strategy:
 Start                     2023-07-05 00:00:00
End                       2024-12-30 00:00:00
Duration                    544 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  30355.54437
Equity Peak [$]                   32859.08838
Return [%]                          203.55544
Buy & Hold Return [%]               225.05234
Return (Ann.) [%]                   110.47576
Volatility (Ann.) [%]               104.96377
CAGR [%]                             67.25907
Sharpe Ratio                          1.05251
Sortino Ratio                          3.7376
Calmar Ratio                          4.09938
Max. Drawdown [%]                    -26.9494
Avg. Drawdown [%]                    -5.86681
Max. Drawdown Duration      118 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst

In [14]:
stats1._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag
0,10,12,224,45.767228,102.500000,None,102.5,567.327724,1.239594,2023-07-21,2024-05-23,307 days,Hello2
1,10,12,224,45.767228,105.000000,None,105.0,592.327724,1.294218,2023-07-21,2024-05-23,307 days,Hello
2,10,11,224,46.485895,102.500000,None,102.5,560.141045,1.204970,2023-07-20,2024-05-23,308 days,Hello2
3,10,11,224,46.485895,105.000000,None,105.0,585.141045,1.258750,2023-07-20,2024-05-23,308 days,Hello
4,10,10,224,47.442467,102.500000,None,102.5,550.575334,1.160512,2023-07-19,2024-05-23,309 days,Hello2
5,10,10,224,47.442467,105.000000,None,105.0,575.575334,1.213207,2023-07-19,2024-05-23,309 days,Hello
6,10,9,224,46.679815,102.500000,None,102.5,558.201855,1.195810,2023-07-18,2024-05-23,310 days,Hello2
7,10,9,224,46.679815,105.000000,None,105.0,583.201855,1.249366,2023-07-18,2024-05-23,310 days,Hello
8,10,8,224,46.267999,102.500000,None,102.5,562.320005,1.215354,2023-07-17,2024-05-23,311 days,Hello2
9,10,8,224,46.267999,105.000000,None,105.0,587.320005,1.269387,2023-07-17,2024-05-23,311 days,Hello
